In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

from PfyMU.gait.train_classifier.core import load_datasets

In [3]:
gait_sets_path = Path('/Users/adamol/Documents/Datasets/gait/processed')

datasets = [
    gait_sets_path / 'bluesky2',
    gait_sets_path / 'daliac',
    gait_sets_path / 'ltmm',
    gait_sets_path / 'usc-had'
]

X, subjects, Y = load_datasets(datasets, goal_fs=50.0, acc_mag=True, window_length=3.0, window_step=0.5)

## Dataset class summary

In [4]:
print('Total samples (3.0s windows): ', Y.size)
print('Total walking samples: ', Y.sum())
print('Total non-walking samples: ', Y.size - Y.sum(), '\n')
print(f'% walking samples: {Y.sum() / Y.size * 100:.2f}')

Total samples (3.0s windows):  82308
Total walking samples:  15345
Total non-walking samples:  66963 

% walking samples: 18.64


## Feature Generation

In [5]:
from PfyMU.features import *

In [6]:
FB = Bank(window_length=None, window_step=None)

# add features
FB + Mean()
FB + MeanCrossRate()
FB + StdDev()
FB + Skewness()
FB + Kurtosis()
FB + Range()
FB + IQR()
FB + RMS()
FB + LinearSlope()
FB + SignalEntropy()
FB + SampleEntropy(m=4, r=1.0)
FB + PermutationEntropy(order=3, delay=1, normalize=True)
FB + ComplexityInvariantDistance(normalize=True)
FB + RangeCountPercentage(range_min=0, range_max=1.0)
FB + RatioBeyondRSigma(r=2.0)
FB + JerkMetric(normalize=True)
FB + DimensionlessJerk(log=True, signal_type='acceleration')
FB + SPARC()
FB + DominantFrequency(low_cutoff=0.25, high_cutoff=5.0)
FB + DominantFrequencyValue(low_cutoff=0.25, high_cutoff=5.0)
FB + PowerSpectralSum(low_cutoff=0.25, high_cutoff=5.0)
FB + SpectralFlatness(low_cutoff=0.25, high_cutoff=5.0)
FB + SpectralEntropy(low_cutoff=0.25, high_cutoff=5.0)

In [7]:
X_feat = FB.compute(X, fs=50.0, windowed=True)

## Feature Exploration

In [8]:
feats = pd.DataFrame(data=X_feat, columns=[i.parent._name for i in FB._feat_list])
feats['Labels'] = Y

### Feature distributions

In [9]:
plt.close('all')

f, ax = plt.subplots(nrows=6, ncols=4, figsize=(10, 10), sharex=True)

j, k = 0, 0
for i, ft in enumerate([i.parent._name for i in FB._feat_list]):
    if k > 3:
        j += 1
        k  = 0
    sns.violinplot(x='Labels', y=ft, data=feats, ax=ax[j, k])
    ax[j, k].set_ylabel(None)
    ax[j, k].set_title(ft)
    ax[j, k].set_xlabel(None)
    
    k += 1

for k in range(4):
    ax[-1, k].set_xticklabels(['NG', 'G'])

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Predictive Power Score

In [10]:
import ppscore

In [11]:
df_predictors = ppscore.predictors(feats, 'Labels', output='df')
plt.figure()
ax = sns.barplot(data=df_predictors, x="x", y="ppscore")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
pps_matrix = ppscore.matrix(feats)

In [13]:
plt.figure(figsize=(13, 10))
sns.heatmap(pps_matrix, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …